In [28]:
import pandas as pd
from sklearn import preprocessing, grid_search
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.grid_search import RandomizedSearchCV
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

In [29]:
startData = pd.read_csv("all_summary.txt", sep=";", index_col=None, na_values=['NA','NaN','','nan'], low_memory=False,
                   keep_default_na = False)

In [30]:
filtered = startData[~startData["res_name"].isin(
        ["DA", "DC", "DT", "DU", "DG", "DI", "UNK", "UNX", "UNL", "PR", "PD", "Y1", "EU", "N", "15P", "UQ", "PX4", "NAN"])]

In [31]:
filtered = filtered[filtered["res_name"].notnull()]

In [32]:
filtered = filtered[filtered.res_name != None]

In [33]:
distinct = filtered.drop_duplicates(subset=["res_name","pdb_code"])

In [34]:
endData = distinct.groupby('res_name').filter(lambda x: len(x) >= 5)

In [35]:
le = preprocessing.LabelEncoder()
le.fit(endData[['res_name']])
endData[['res_name']] = le.transform(endData[['res_name']])

In [36]:
completeColumnEndData = endData.count()==len(endData)
completeColumnEndData = completeColumnEndData.values
completeColumnEndData = endData.iloc[:,completeColumnEndData]
X = completeColumnEndData.select_dtypes(['float64'])
y = completeColumnEndData['res_name']

In [37]:
print type(X)
print X.shape
print type(y)
print y.shape

<class 'pandas.core.frame.DataFrame'>
(10767, 73)
<class 'pandas.core.series.Series'>
(10767L,)


In [253]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.cross_validation import train_test_split
knn = KNeighborsClassifier(n_neighbors=13)
k_range = range(1, 31)
param_grid = dict(n_neighbors=k_range)
knnResult = grid_search.GridSearchCV(knn, param_grid=param_grid, scoring = 'recall')
knnResult.fit(X,y)
knnResult.grid_scores_

[mean: 0.34160, std: 0.00244, params: {'n_neighbors': 1},
 mean: 0.29869, std: 0.00351, params: {'n_neighbors': 2},
 mean: 0.33788, std: 0.00055, params: {'n_neighbors': 3},
 mean: 0.34531, std: 0.00529, params: {'n_neighbors': 4},
 mean: 0.35182, std: 0.01018, params: {'n_neighbors': 5},
 mean: 0.35618, std: 0.00582, params: {'n_neighbors': 6},
 mean: 0.36343, std: 0.00378, params: {'n_neighbors': 7},
 mean: 0.36398, std: 0.00246, params: {'n_neighbors': 8},
 mean: 0.36575, std: 0.00523, params: {'n_neighbors': 9},
 mean: 0.36918, std: 0.00564, params: {'n_neighbors': 10},
 mean: 0.36956, std: 0.00317, params: {'n_neighbors': 11},
 mean: 0.37030, std: 0.00366, params: {'n_neighbors': 12},
 mean: 0.36965, std: 0.00212, params: {'n_neighbors': 13},
 mean: 0.37002, std: 0.00378, params: {'n_neighbors': 14},
 mean: 0.36993, std: 0.00643, params: {'n_neighbors': 15},
 mean: 0.36937, std: 0.00600, params: {'n_neighbors': 16},
 mean: 0.37021, std: 0.00633, params: {'n_neighbors': 17},
 mean:

In [242]:
rf = RandomForestClassifier()

In [258]:
param_grid = {"n_estimators" : range(1, 31),
              "criterion" : ["gini","entropy"]
             }
clf = grid_search.GridSearchCV(rf, param_grid=param_grid, scoring = 'recall_macro')

In [259]:
clf.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, scoring='recall_macro',
       verbose=0)

In [260]:
clf.grid_scores_

[mean: 0.11023, std: 0.01914, params: {'n_estimators': 1, 'criterion': 'gini'},
 mean: 0.10390, std: 0.01772, params: {'n_estimators': 2, 'criterion': 'gini'},
 mean: 0.09899, std: 0.00596, params: {'n_estimators': 3, 'criterion': 'gini'},
 mean: 0.11449, std: 0.00523, params: {'n_estimators': 4, 'criterion': 'gini'},
 mean: 0.12845, std: 0.00947, params: {'n_estimators': 5, 'criterion': 'gini'},
 mean: 0.13093, std: 0.00602, params: {'n_estimators': 6, 'criterion': 'gini'},
 mean: 0.15285, std: 0.00881, params: {'n_estimators': 7, 'criterion': 'gini'},
 mean: 0.14529, std: 0.00491, params: {'n_estimators': 8, 'criterion': 'gini'},
 mean: 0.14107, std: 0.01080, params: {'n_estimators': 9, 'criterion': 'gini'},
 mean: 0.15637, std: 0.01722, params: {'n_estimators': 10, 'criterion': 'gini'},
 mean: 0.15908, std: 0.00746, params: {'n_estimators': 11, 'criterion': 'gini'},
 mean: 0.15714, std: 0.01462, params: {'n_estimators': 12, 'criterion': 'gini'},
 mean: 0.16500, std: 0.01160, params:

In [263]:
param_grid = {"n_estimators" : range(1, 31),
              "criterion" : ["gini","entropy"],
              "max_depth" : range(1, 5),
              "max_features" : [0.05,0.1,0.2,"log2","auto",None]
             }
rand = RandomizedSearchCV(rf, param_grid, cv=10, scoring='recall_macro', n_iter=10, random_state=5)

In [264]:
rand.fit(X,y)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'max_features': [0.05, 0.1, 0.2, 'log2', 'auto', None], 'criterion': ['gini', 'entropy'], 'max_depth': [1, 2, 3, 4]},
          pre_dispatch='2*n_jobs', random_state=5, refit=True,
          scoring='recall_macro', verbose=0)

In [265]:
rand.grid_scores_

[mean: 0.01026, std: 0.00280, params: {'n_estimators': 28, 'max_features': 'auto', 'criterion': 'entropy', 'max_depth': 1},
 mean: 0.01653, std: 0.00582, params: {'n_estimators': 27, 'max_features': 0.05, 'criterion': 'gini', 'max_depth': 2},
 mean: 0.01759, std: 0.00619, params: {'n_estimators': 9, 'max_features': 'log2', 'criterion': 'entropy', 'max_depth': 2},
 mean: 0.02943, std: 0.00776, params: {'n_estimators': 3, 'max_features': 0.2, 'criterion': 'entropy', 'max_depth': 3},
 mean: 0.08507, std: 0.02338, params: {'n_estimators': 15, 'max_features': None, 'criterion': 'entropy', 'max_depth': 4},
 mean: 0.01431, std: 0.00356, params: {'n_estimators': 14, 'max_features': 0.2, 'criterion': 'gini', 'max_depth': 1},
 mean: 0.02018, std: 0.00559, params: {'n_estimators': 13, 'max_features': 'auto', 'criterion': 'entropy', 'max_depth': 2},
 mean: 0.00685, std: 0.00193, params: {'n_estimators': 21, 'max_features': 0.05, 'criterion': 'entropy', 'max_depth': 1},
 mean: 0.03625, std: 0.00848

In [11]:
grupedLab = pd.read_csv("grouped_res_name.txt", sep=",", index_col=None, na_values=['NA','NaN','','nan'], low_memory=False,
                   keep_default_na = False)
endDataG = endData
endDataG['res_name']=grupedLab['res_name_group']

In [14]:
rfG = RandomForestClassifier()

In [17]:
param_gridG = {"n_estimators" : [29],
              "criterion" : ["entropy"]
             }
GSrfG = grid_search.GridSearchCV(rfG, param_grid=param_gridG, scoring = 'recall_macro')

In [18]:
GSrfG.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [29], 'criterion': ['entropy']},
       pre_dispatch='2*n_jobs', refit=True, scoring='recall_macro',
       verbose=0)

In [19]:
GSrfG.grid_scores_

[mean: 0.15847, std: 0.00323, params: {'n_estimators': 29, 'criterion': 'entropy'}]

In [21]:
param_gridG = {"n_estimators" : range(10, 40),
              "criterion" : ["gini","entropy"],
              "max_depth" : range(4, 10),
              "max_features" : [0.05,0.1,0.2,"log2","auto",None]
             }
randG = RandomizedSearchCV(rfG, param_gridG, cv=10, scoring='recall_macro', n_iter=10, random_state=5)

In [22]:
randG.fit(X,y)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 'max_features': [0.05, 0.1, 0.2, 'log2', 'auto', None], 'criterion': ['gini', 'entropy'], 'max_depth': [4, 5, 6, 7, 8, 9]},
          pre_dispatch='2*n_jobs', random_state=5, refit=True,
          scoring='recall_macro', verbose=0)

In [23]:
randG.grid_scores_

[mean: 0.07899, std: 0.01671, params: {'n_estimators': 25, 'max_features': 'log2', 'criterion': 'entropy', 'max_depth': 7},
 mean: 0.16558, std: 0.03630, params: {'n_estimators': 24, 'max_features': None, 'criterion': 'entropy', 'max_depth': 5},
 mean: 0.14793, std: 0.03283, params: {'n_estimators': 31, 'max_features': 'auto', 'criterion': 'entropy', 'max_depth': 9},
 mean: 0.11116, std: 0.01995, params: {'n_estimators': 22, 'max_features': 'auto', 'criterion': 'gini', 'max_depth': 9},
 mean: 0.06186, std: 0.01250, params: {'n_estimators': 30, 'max_features': 0.05, 'criterion': 'gini', 'max_depth': 8},
 mean: 0.08375, std: 0.01923, params: {'n_estimators': 12, 'max_features': 0.05, 'criterion': 'entropy', 'max_depth': 9},
 mean: 0.03851, std: 0.00719, params: {'n_estimators': 34, 'max_features': 0.05, 'criterion': 'gini', 'max_depth': 5},
 mean: 0.04330, std: 0.00830, params: {'n_estimators': 31, 'max_features': 0.05, 'criterion': 'entropy', 'max_depth': 6},
 mean: 0.11875, std: 0.0258

In [38]:
param_gridG = {"n_estimators" : [31],
              "criterion" : ["gini"],
              "max_depth" : [11],
              "max_features" : [None]
             }
randG2 = RandomizedSearchCV(rfG, param_gridG, cv=10, scoring='recall_macro', n_iter=1, random_state=5)

In [39]:
randG2.fit(X,y)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params={}, iid=True, n_iter=1, n_jobs=1,
          param_distributions={'n_estimators': [31], 'max_features': [None], 'criterion': ['gini'], 'max_depth': [11]},
          pre_dispatch='2*n_jobs', random_state=5, refit=True,
          scoring='recall_macro', verbose=0)

In [40]:
randG2.grid_scores_

[mean: 0.41333, std: 0.04395, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 11}]

In [41]:
param_gridG = {"n_estimators" : [31],
              "criterion" : ["gini"],
              "max_depth" : [13],
              "max_features" : [None]
             }
randG3 = RandomizedSearchCV(rfG, param_gridG, cv=10, scoring='recall_macro', n_iter=1, random_state=5)

In [42]:
randG3.fit(X,y)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params={}, iid=True, n_iter=1, n_jobs=1,
          param_distributions={'n_estimators': [31], 'max_features': [None], 'criterion': ['gini'], 'max_depth': [13]},
          pre_dispatch='2*n_jobs', random_state=5, refit=True,
          scoring='recall_macro', verbose=0)

In [43]:
randG3.grid_scores_

[mean: 0.49008, std: 0.05489, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 13}]

In [44]:
param_gridG = {"n_estimators" : [31],
              "criterion" : ["gini"],
              "max_depth" : [15],
              "max_features" : [None]
             }
randG4 = RandomizedSearchCV(rfG, param_gridG, cv=10, scoring='recall_macro', n_iter=1, random_state=5)
randG4.fit(X,y)
randG4.grid_scores_

[mean: 0.58532, std: 0.05372, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 15}]

In [45]:
param_gridG = {"n_estimators" : [31],
              "criterion" : ["gini"],
              "max_depth" : [17],
              "max_features" : [None]
             }
randG5 = RandomizedSearchCV(rfG, param_gridG, cv=10, scoring='recall_macro', n_iter=1, random_state=5)
randG5.fit(X,y)
randG5.grid_scores_

[mean: 0.64643, std: 0.03839, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 17}]

In [46]:
param_gridG = {"n_estimators" : [31],
              "criterion" : ["gini"],
              "max_depth" : [23],
              "max_features" : [None]
             }
randG6 = RandomizedSearchCV(rfG, param_gridG, cv=10, scoring='recall_macro', n_iter=1, random_state=5)
randG6.fit(X,y)
randG6.grid_scores_

[mean: 0.67589, std: 0.02389, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 23}]

In [47]:
param_gridG = {"n_estimators" : [31],
              "criterion" : ["gini"],
              "max_depth" : [27],
              "max_features" : [None]
             }
randG7 = RandomizedSearchCV(rfG, param_gridG, cv=10, scoring='recall_macro', n_iter=1, random_state=5)
randG7.fit(X,y)
randG7.grid_scores_

[mean: 0.66867, std: 0.01727, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 27}]